In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import ensemble,model_selection,preprocessing,tree

In [21]:
data_bit = pd.read_csv('data/bitcoin_timestamp.csv')

In [22]:
"""
Volume_(Currency) is dollars
predictive model that allows us to buy or sell bitcoin 
"""

data_bit.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
2,1325318040,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
3,1325318100,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
4,1325318160,4.39,4.39,4.39,4.39,0.455581,2.0,4.39


In [23]:
data_bit.shape

(21881, 8)

In [24]:
"""
Timestamp is an int. everything else is a float
"""
data_bit.dtypes

Timestamp              int64
Open                 float64
High                 float64
Low                  float64
Close                float64
Volume_(BTC)         float64
Volume_(Currency)    float64
Weighted_Price       float64
dtype: object

In [25]:
"""
this is now a datatime series
"""
pd.to_datetime(data_bit.Timestamp, unit='s')

0       2011-12-31 07:52:00
1       2011-12-31 07:53:00
2       2011-12-31 07:54:00
3       2011-12-31 07:55:00
4       2011-12-31 07:56:00
5       2011-12-31 07:57:00
6       2011-12-31 07:58:00
7       2011-12-31 07:59:00
8       2011-12-31 08:00:00
9       2011-12-31 08:01:00
10      2011-12-31 08:02:00
11      2011-12-31 08:03:00
12      2011-12-31 08:04:00
13      2011-12-31 08:05:00
14      2011-12-31 08:06:00
15      2011-12-31 08:07:00
16      2011-12-31 08:08:00
17      2011-12-31 08:09:00
18      2011-12-31 08:10:00
19      2011-12-31 08:11:00
20      2011-12-31 08:12:00
21      2011-12-31 08:13:00
22      2011-12-31 08:14:00
23      2011-12-31 08:15:00
24      2011-12-31 08:16:00
25      2011-12-31 08:17:00
26      2011-12-31 08:18:00
27      2011-12-31 08:19:00
28      2011-12-31 08:20:00
29      2011-12-31 08:21:00
                ...        
21851   2012-01-15 12:03:00
21852   2012-01-15 12:04:00
21853   2012-01-15 12:05:00
21854   2012-01-15 12:06:00
21855   2012-01-15 1

In [26]:
data_bit['date'] = pd.to_datetime(data_bit.Timestamp, unit='s')

In [28]:
%time
data_bit.head(2)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs


,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,date
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31 07:52:00
1,1325317980,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31 07:53:00


In [30]:
data_bit.columns

Index(['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume_(BTC)',
       'Volume_(Currency)', 'Weighted_Price', 'date'],
      dtype='object')

In [48]:
bitcoin_data = data_bit.copy()

In [49]:
bitcoin_data.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,date
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31 07:52:00
1,1325317980,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31 07:53:00
2,1325318040,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31 07:54:00
3,1325318100,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31 07:55:00
4,1325318160,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31 07:56:00


In [60]:
 """
this now has a data index
data is no longer on minute granularity. It's now on day
"""
bit_df = (
bitcoin_data.set_index('date')
    .rename(columns={'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume_(BTC)':'vol_btc',
       'Volume_(Currency)':'vol_cur', 'Weighted_Price': 'wp','Timestamp':'ts'})
    .resample('d') 
    .agg({'open':'first','high':'mean','low':'mean', 'close':'last','vol_btc':'sum','vol_cur':'sum','wp':'mean', 'ts':'min'})
    
)   

In [61]:
bit_df.columns

Index(['open', 'high', 'low', 'close', 'vol_btc', 'vol_cur', 'wp', 'ts'], dtype='object')

In [62]:
bit_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16 entries, 2011-12-31 to 2012-01-15
Freq: D
Data columns (total 8 columns):
open       16 non-null float64
high       16 non-null float64
low        16 non-null float64
close      16 non-null float64
vol_btc    16 non-null float64
vol_cur    16 non-null float64
wp         16 non-null float64
ts         16 non-null int64
dtypes: float64(7), int64(1)
memory usage: 1.8 KB


In [82]:
"""
make column for buying, using shift operator
everything enclosed moved up 1
is the close tomorrow better or worse than today
is the closr of the next day, greater than close of today
buy or sell based on close
"""
bit_buy = bit_df.assign(buy = lambda x: (x.close.shift(-1) > x.close).astype(int))

In [83]:
bit_buy

,open,high,low,close,vol_btc,vol_cur,wp,ts,buy
date,,,,,,,,,
2011-12-31,4.39,4.472624,4.472552,4.58,7347.629953,32979.460347,4.472579,1325317920,1
2012-01-01,4.58,4.680778,4.680778,5.00,8440.844000,40113.415520,4.680778,1325376000,0
2012-01-02,5.00,5.000000,5.000000,5.00,16655.728000,83278.640000,5.000000,1325462400,1
2012-01-03,5.00,5.145917,5.145917,5.29,19448.819802,98964.563352,5.145917,1325548800,1
2012-01-04,5.29,5.228729,5.176708,5.57,24798.296265,131938.688442,5.195443,1325635200,1
2012-01-05,5.37,6.134917,6.116778,6.65,9102.080979,51820.276320,6.124426,1325721600,0
2012-01-06,6.65,6.551583,6.531069,6.00,5331.130797,35437.650812,6.540973,1325808000,1
2012-01-07,6.00,6.192222,6.192222,6.80,1196.366865,7260.094679,6.192222,1325894400,1
2012-01-08,6.80,6.842292,6.842292,7.00,2487.437871,17140.777526,6.842292,1325980800,0


Random Forest

In [85]:
"""
make a bunch of deicion trees, is this true, if yes
   at teh end of path, should you buy or sell bitcoin

"""
X = bit_buy.drop('buy',axis=1)
y = bit_buy['buy']

In [86]:
print(X.shape)
print(y.shape)

(16, 8)
(16,)


In [87]:
"""
should I buy or sell that day
1 = buy
0=sell

use X to make a model to predcit whether we should buy or sell.
"""
y

date
2011-12-31    1
2012-01-01    0
2012-01-02    1
2012-01-03    1
2012-01-04    1
2012-01-05    0
2012-01-06    1
2012-01-07    1
2012-01-08    0
2012-01-09    1
2012-01-10    0
2012-01-11    0
2012-01-12    1
2012-01-13    0
2012-01-14    1
2012-01-15    0
Freq: D, Name: buy, dtype: int64

In [89]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [91]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(10, 8)
(10,)
(6, 8)
(6,)


In [108]:
"""
train subset is overfitting, it's not generalizing to data it hasn't seen before.

score is the R2 method
   returns the mean accuracy on the test data and labels
"""

from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier(random_state=42)
rf1.fit(X_train,y_train)
print('Score on train subset: {}'.format(rf1.score(X_train,y_train)))
print('Score on train subset: {}'.format(rf1.score(X_test,y_test)))     

Score on train subset: 1.0
Score on train subset: 0.3333333333333333


In [ ]:
#ROCAUC curve
